In [1]:
!pip install simpletransformers
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from typing import Dict, List, Any
import logging
from sklearn.preprocessing import LabelBinarizer,LabelEncoder
import jsonlines
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import random 
import numpy as np
import torch

def random_seed(seed_value): 
    np.random.seed(seed_value) 
    torch.manual_seed(seed_value)
    random.seed(seed_value) 


random_seed(42)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.8/249.8 kB 492.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 8.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 17.9 MB/s eta 0:00:0000:0100:01
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=ec554e12d059a7941555df1752c4bf664da7a332d7a9a37da393fb0f1b84eb56
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=2b40cbfa341789486c9fc8f40aeabefb00e031044f1ec5b4aaf52621a5173525
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9ca

In [2]:
def read_give_instances(path):
    
    texts = []
    labels = []
    with jsonlines.open(path) as f_in:
        for json_object in f_in:
            text=json_object.get('text')
            label=json_object.get('label')
            metadata=json_object.get('metadata')
            texts.append(text)
            labels.append(label)
            
    return texts, labels

In [3]:
train_texts, train_labels = read_give_instances("../input/scibert-dataset/text_classification/sciie-relation-extraction/train.txt")
dev_texts, dev_labels = read_give_instances("../input/scibert-dataset/text_classification/sciie-relation-extraction/dev.txt")
test_texts, test_labels = read_give_instances("../input/scibert-dataset/text_classification/sciie-relation-extraction/test.txt")

In [4]:
np.unique(train_labels)

array(['COMPARE', 'CONJUNCTION', 'EVALUATE-FOR', 'FEATURE-OF',
       'HYPONYM-OF', 'PART-OF', 'USED-FOR'], dtype='<U12')

In [5]:
train_texts[4], train_labels[:5]

('In this paper , a novel method to learn the [[ intrinsic object structure ]] for << robust visual tracking >> is proposed .',
 ['CONJUNCTION', 'FEATURE-OF', 'HYPONYM-OF', 'USED-FOR', 'USED-FOR'])

In [6]:
df_train = pd.DataFrame({"text":train_texts,"labels":train_labels})
df_val = pd.DataFrame({"text":dev_texts,"labels":dev_labels})
df_test = pd.DataFrame({"text":test_texts,"labels":test_labels})
df_train.head()

,text,labels
0,The agreement in question involves number in [...,CONJUNCTION
1,The agreement in question involves number in n...,FEATURE-OF
2,The agreement in question involves number in n...,HYPONYM-OF
3,"In this paper , a novel [[ method ]] to learn ...",USED-FOR
4,"In this paper , a novel method to learn the [[...",USED-FOR


In [7]:
len(df_train['labels'].unique())

7

In [8]:
lb = LabelEncoder()

df_train['labels'] = lb.fit_transform(df_train['labels'])
df_val['labels'] = lb.transform(df_val['labels'])
df_test['labels'] = lb.transform(df_test['labels'])
df_train.head()

,text,labels
0,The agreement in question involves number in [...,1
1,The agreement in question involves number in n...,3
2,The agreement in question involves number in n...,4
3,"In this paper , a novel [[ method ]] to learn ...",6
4,"In this paper , a novel method to learn the [[...",6


In [9]:
len(df_train['labels'].unique())

7

In [10]:
model_args = ClassificationArgs(eval_batch_size=64,
                               learning_rate = 2e-5,
                               max_seq_length = 256,
                               no_save = True,
                               num_train_epochs = 4,
                               train_batch_size = 32)

In [11]:
model = ClassificationModel("bert","AnonymousSub/fpdm_models_scibert_hybrid_epochs_4",args = model_args,num_labels=7)

Downloading:   0%|          | 0.00/706 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at AnonymousSub/fpdm_models_scibert_hybrid_epochs_4 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/427 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/223k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [12]:
model.train_model(df_train,eval_df = df_val)

  0%|          | 0/3219 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/101 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/101 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/101 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/101 [00:00<?, ?it/s]

(404, 0.6215795094497724)

In [13]:
predictions, raw_outputs = model.predict(df_test['text'].tolist())

  0%|          | 0/974 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [14]:
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, accuracy_score

In [15]:
print(classification_report(df_test['labels'], predictions))

              precision    recall  f1-score   support

           0       0.80      0.74      0.77        38
           1       0.90      0.96      0.93       123
           2       0.81      0.87      0.84        91
           3       0.59      0.68      0.63        59
           4       0.88      0.96      0.91        67
           5       0.62      0.44      0.52        63
           6       0.94      0.92      0.93       533

    accuracy                           0.87       974
   macro avg       0.79      0.79      0.79       974
weighted avg       0.87      0.87      0.87       974



In [17]:
print(accuracy_score(df_test['labels'], predictions))
print(precision_score(df_test['labels'], predictions,average='macro'))
print(recall_score(df_test['labels'], predictions,average='macro'))
print(f1_score(df_test['labels'], predictions,average='macro'))

0.8696098562628337
0.789881450954572
0.7944689387840712
0.7888733989207618
